In [4]:
import warnings
warnings.filterwarnings('ignore')


In [3]:


from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from IPython.display import JSON
import json
from markdownify import markdownify as md

client = UnstructuredClient(
    api_key_auth="xNQcZcO2UAju3p5Kd9pZjTMnqf9zzr",
    # server_url=DLAI_API_URL,
)

filename = "Pmg_lds.pdf"
with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(), 
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy='hi_res',
    pdf_infer_table_structure=True,
    languages=["eng"],
    
)


# Partition the PDF file
resp = client.general.partition(req)


INFO: Preparing to split document for partition.
INFO: Starting page number set to 1
INFO: Concurrency level set to 5
INFO: Determined optimal split size of 20 pages.
INFO: Document split into 13, 20-paged sets.
INFO: Partitioning 13, 20-paged sets.
INFO: Partitioning set #1 (pages 1-21).
INFO: Partitioning set #2 (pages 21-41).
INFO: Partitioning set #3 (pages 41-61).
INFO: Partitioning set #4 (pages 61-81).
INFO: Partitioning set #5 (pages 81-101).
INFO: Partitioning set #6 (pages 101-121).
INFO: Partitioning set #7 (pages 121-141).
INFO: Partitioning set #8 (pages 141-161).
INFO: Partitioning set #9 (pages 161-181).
INFO: Partitioning set #10 (pages 181-201).
INFO: Partitioning set #11 (pages 201-221).
INFO: Partitioning set #12 (pages 221-241).
INFO: Partitioning set #13 (pages 241-244).
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Reques

In [54]:
print(json.dumps(resp.elements,indent=2))

[
  {
    "type": "Image",
    "element_id": "901619d822b1fd84233b99cf7dbf2083",
    "text": "A Guide to Missionary Service  Preach My Gospel  (D&C 50:14) ",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "filename": "Pmg_lds.pdf"
    }
  },
  {
    "type": "Title",
    "element_id": "0037e5e74d4951594c8c8a99714dce52",
    "text": "A Guide to Missionary Service",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "filename": "Pmg_lds.pdf"
    }
  },
  {
    "type": "Title",
    "element_id": "c7b9347d5636f1bd64df9179ff2a5cd1",
    "text": "Preach My Gospel",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "filename": "Pmg_lds.pdf"
    }
  },
  {
    "type": "UncategorizedText",
    "element_id": "d0dfb6cb26ac0e4d35fe4c41c55ec8f7",
    "text": "(D&C 

In [53]:
# JSON(json.dumps(resp.elements, indent=2))
keys = set()
for element in resp.elements:
    keys.add(element['type'])
print(keys)



{'Header', 'UncategorizedText', 'PageNumber', 'ListItem', 'NarrativeText', 'Title', 'Table', 'Image'}


In [56]:
# convert dictionary elements to html tags - transform to markdown
# with markdownify

def format_to_html(content):
    if content['type'] == 'Header':
        header = content['text']
        return f"<h2>{header}</h2>\n\n"
    
    elif content['type'] == 'UncategorizedText':
        return f"<span>{content['text']}</span>\n\n"
    
    elif content['type'] == 'PageNumber':
        page_number = content['text']
        return f"<span>{page_number}</span>\n\n"
    
    elif content['type'] == 'ListItem':
        list_items = content['text']
        return f"<ul><li>{list_items}</li></ul>\n\n"
    
    elif content['type'] == 'NarrativeText':
        return f"<p>{content['text']}</p>\n\n"
    
    elif content['type'] == 'Title':
        return f"<h1>{content['text']}</h1>\n\n"
    
    elif content['type'] == 'Table':
        value = content['metadata']
        return f"{value['text_as_html']}\n\n"
    
    elif content['type'] == 'Image':
        alt_text = content['metadata']
        return f"<img src='#' alt='see page {alt_text['page_number']}' />\n\n"
    
    else:
        return ""


# function to write to markdown file
def save_to_markdown(text, markdown_file):
    with open(markdown_file, 'w', encoding='utf-8') as f:
        f.writelines(text)


In [57]:
# iterate through the response and format
html_tags = ""

for element in resp.elements:
    html_tags += format_to_html(element)

#convert to markdown using markdownify
md_content = md(html_tags)

# save to markdown file
save_to_markdown(md_content, 'Pmg_lds.md')
